 ## Exercise 46 - Time Series Analysis

In [1]:
inputPath = "/data/students/bigdata-01QYD/ex_data/Ex46/data/readings.txt"
outputPath = "res_out_Ex46/"
threshold = 0.5

__input__ : 
* A textual file containing a set of temperature readings : _timestamp (UNIX), temperature_, sample rate = 1 min

__output__ : 
*  Consider all the windows containing 3 consecutive temperature readings and
    * Select the windows characterized by an increasing trend (temperature(t)>temperature(t-60 seconds))
    * Store the result into an HDFS file

In [2]:
# map into (movieid , userid)
readingsRDD = sc.textFile(inputPath)
readingsRDD.collect()

['1451606400,12.1',
 '1451606460,12.2',
 '1451606520,13.5',
 '1451606580,14.0',
 '1451606640,14.0',
 '1451606700,15.5',
 '1451606760,15.0']

In [3]:
# create a window of three elements
# reading at time t, t-60, 
def windowReadingsFunction(line):
    
    temp = float(line.split(",")[1])
    reading = int(line.split(",")[0])
    
    # collect the pairs (reading,temp)
    window = []
    
    window.append((reading,line))
    window.append((reading-60,line))
    window.append((reading-120,line))
    
    return window


windowsPairsRDD = readingsRDD.flatMap(windowReadingsFunction)
# group by each timestamp
# in order to create the window
timestampsWindowsRDD = windowsPairsRDD.groupByKey()

In [4]:
timestampsWindowsRDD.mapValues(lambda v: list(v)).collect()

[(1451606400, ['1451606400,12.1', '1451606460,12.2', '1451606520,13.5']),
 (1451606340, ['1451606400,12.1', '1451606460,12.2']),
 (1451606280, ['1451606400,12.1']),
 (1451606460, ['1451606460,12.2', '1451606520,13.5', '1451606580,14.0']),
 (1451606520, ['1451606520,13.5', '1451606580,14.0', '1451606640,14.0']),
 (1451606580, ['1451606580,14.0', '1451606640,14.0', '1451606700,15.5']),
 (1451606640, ['1451606640,14.0', '1451606700,15.5', '1451606760,15.0']),
 (1451606700, ['1451606700,15.5', '1451606760,15.0']),
 (1451606760, ['1451606760,15.0'])]

In [15]:
# filter just the increasing windows
def getIncreasingWindows(elements):
    
    timestamp, window = elements
    
    # memorize tuple (timestamp, temp)
    temperatures = []
    
    for t in window:
        temperatures.append( ( int(t.split(",")[0]), float(t.split(",")[1])) )
        
    # filter increasing trends
    if len(temperatures) < 3:
        return False
    
    else:
        if temperatures[0][1] < temperatures[1][1] and \
           temperatures[1][1] < temperatures[2][1]:
            return True
        else:
            return False

In [17]:
filteredWindows = timestampsWindowsRDD.filter(getIncreasingWindows)
filteredWindows.mapValues(lambda iterable : list(iterable)).values().collect()

[['1451606400,12.1', '1451606460,12.2', '1451606520,13.5'],
 ['1451606460,12.2', '1451606520,13.5', '1451606580,14.0']]

In [19]:
filteredWindows.mapValues(lambda iterable : list(iterable)).values().saveAsTextFile(outputPath)